In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [2]:
import sys
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out import create_dir

from geo_tool import Point_Cloud

from tf_lab.point_clouds.ae_templates import mlp_architecture_ala_iclr_18, default_train_params
from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.convenience import reconstruct_pclouds

from tf_lab.data_sets.shape_net import snc_category_to_synth_id
from tf_lab.iclr.helper import load_multiple_version_of_pcs, find_best_validation_epoch_from_train_stats

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
top_data_dir = '/orions4-zfs/projects/optas/DATA/'

experiment_tag =  'mlp_with_split_1pc_usampled_bnorm_on_encoder_only_v2'

n_pc_points = 2048

random_seed = 42

loss = 'emd'

class_name = raw_input('Give me the class type: ').lower()

syn_id = snc_category_to_synth_id()[class_name]

Give me the class type: chair


In [5]:
in_data = load_multiple_version_of_pcs('uniform_one', syn_id, n_classes=1)

Loading test data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/test.txt
679 pclouds were loaded. They belong in 1 shape-classes.
Loading train data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/train.txt
5761 pclouds were loaded. They belong in 1 shape-classes.
Loading val data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/val.txt
338 pclouds were loaded. They belong in 1 shape-classes.


In [6]:
bneck_list = [4, 8, 16, 32, 64, 128, 256, 512]

for bneck in bneck_list:
    experiment_id = '_'.join(['ae', class_name, experiment_tag, str(n_pc_points), 'pts', str(bneck), 'bneck', loss])
    train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/', experiment_id)
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    
    val_error, best_epoch = find_best_validation_epoch_from_train_stats(osp.join(train_dir, 'train_stats.txt'))
    
    if best_epoch % conf.saver_step != 0: # Model was not saved at that epoch: use next bigger epoch.
        epoch_to_load = best_epoch + (best_epoch % conf.saver_step)
    else:
        epoch_to_load = best_epoch
    
    conf.encoder_args['verbose'] = False
    conf.decoder_args['verbose'] = False    
    reset_tf_graph()
    ae = PointNetAutoEncoder(conf.experiment_name, conf)    
    ae.restore_model(conf.train_dir, epoch_to_load, verbose=False)
    _, l_val = reconstruct_pclouds(ae, in_data['val'].point_clouds, batch_size=100)    
    _, l_test = reconstruct_pclouds(ae, in_data['test'].point_clouds, batch_size=100)
    _, l_train = reconstruct_pclouds(ae, in_data['train'].point_clouds, batch_size=100)
    print bneck, l_val, l_test, l_train, best_epoch

4 148.091099666 148.581416426 135.641225093 470
8 130.597838543 132.221149136 112.592633653 500
16 111.417881892 113.98936259 97.2969486453 450
32 103.846909427 106.220607662 92.3266792873 380
64 98.4956156714 101.752321684 87.586505921 450
128 98.7875973402 102.042568269 88.5168912676 425
256 98.6771176581 100.740916964 88.5448542831 405
512 102.962270342 104.733981642 93.5644253289 495
